In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
import os
import pickle
import re
import sys
import logging
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder
from gensim.models import Word2Vec
import gensim

from numpy import *
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import *
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from MultTensorDataset import MultTensorDataset
from LSTM_MultID_Model import ADEM_Age_Model,ADLSTM_Gender__Model

train_path = 'data/train_preliminary/'
test_path = 'data/test/'
id_seq_path = './id_seq_path/'
w2v_model_path = './w2v_model/'
# feat = ['creative_id','ad_id','advertiser_id','product_id','product_category','industry']
feat = ['creative_id','ad_id','advertiser_id','product_id']
model_age_path = './pred_model/bilstm_age_model_nodropout22_3.pth'
model_gender_path = './pred_model/mult_bilstm_gender_model4.pth'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def load_w2v(feat):
    print(w2v_model_path+'word2vec_'+ feat +'_128_win100.model')
    w2v_model = Word2Vec.load(w2v_model_path+'word2vec_'+ feat +'_128_win100.model')
    print(w2v_model)
    vocab_list = [word for word, Vocab in w2v_model.wv.vocab.items()]
#     word_index = {" ": 0} 
#     word_vector = {}
    embedding_matrix = np.zeros((len(vocab_list) + 1, w2v_model.vector_size))
    for i in range(len(vocab_list)):
        word = vocab_list[i]  # 每个词语
#         word_index[word] = i + 1 # 词语：索引
#         word_vector[word] = w2v_model.wv[word] # 词语：词向量
        embedding_matrix[i + 1] = w2v_model.wv[word]  # 词向量矩阵
    print(embedding_matrix.shape)
    return embedding_matrix

In [3]:
df_seq_data_test = pd.read_csv(id_seq_path+'df_all_id_seq_test.csv')
for f in feat:
    df_seq_data_test[f+'_int_seq'] = df_seq_data_test[f+'_int_seq'].apply(lambda x: eval(x))
df_seq_data_test

,Unnamed: 0,user_id,creative_id_int_seq,ad_id_int_seq,advertiser_id_int_seq,product_id_int_seq,product_category_int_seq,industry_int_seq
0,0,3000001,"[378824, 1090, 31066, 5110, 10465, 61587, 2881...","[554955, 41877, 7301, 163, 4000, 117146, 16695...","[289, 457, 599, 600, 457, 5620, 2000, 781, 119...","[3, 3, 4, 31, 3, 2780, 3, 290, 594, 2067, 3]","[1, 1, 9, 9, 1, 9, 1, 2, 2, 4, 1]","[17, 17, 21, 17, 17, 3, 17, 20, 3, 34, 17]"
1,1,3000002,"[136203, 3922, 11663, 214930, 169236, 405, 619...","[26266, 9946, 42102, 232896, 121836, 3303, 455...","[1177, 3196, 44, 871, 3267, 222, 8, 542, 2344,...","[59, 70, 385, 3, 3, 26, 3, 23, 81, 3, 304, 66,...","[2, 2, 2, 10, 4, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1,...","[1, 58, 45, 9, 91, 45, 58, 17, 23, 1, 39, 9, 3..."
2,2,3000003,"[33181, 14030, 171368, 14030, 100008, 310623, ...","[10513, 44378, 33454, 44378, 95022, 288805, 10...","[1416, 3532, 259, 3532, 8, 2254, 2214, 4417, 8...","[24, 571, 37, 571, 3, 5742, 769, 3555, 3, 3, 3...","[2, 2, 2, 2, 1, 4, 2, 4, 1, 1, 1, 1, 2, 2, 2, ...","[21, 21, 23, 21, 1, 97, 19, 33, 1, 1, 13, 1, 1..."
3,3,3000004,"[7287, 1378373, 625, 6662, 599392, 2385, 13783...","[8953, 1563990, 285, 1236, 311292, 268, 224348...","[381, 1165, 65, 161, 6956, 65, 482, 2496, 635,...","[58, 3, 3, 68, 3, 3, 3, 81, 326, 3, 326, 3, 66...","[2, 1, 1, 2, 4, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, ...","[23, 7, 15, 23, 10, 15, 13, 23, 45, 73, 45, 13..."
4,4,3000005,"[437489, 311410, 584943, 1397, 185248, 2444234...","[550264, 878326, 150761, 21643, 546776, 126708...","[11683, 2080, 2444, 52, 4329, 9949, 53, 3253, ...","[3, 3, 3, 3, 429, 70, 70, 646, 59, 3, 913, 57,...","[1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, ...","[46, 39, 104, 13, 39, 1, 1, 3, 1, 73, 19, 23, ..."
...,...,...,...,...,...,...,...,...
999995,999995,3999996,"[4109, 160, 1012, 32084, 1185877, 1768, 118587...","[5614, 513, 440, 4702, 1690777, 27672, 361987,...","[4284, 203, 10, 53, 5259, 1107, 3675, 1678, 30...","[336, 3, 3, 70, 62, 44, 2295, 258, 304, 66, 3,...","[2, 7, 7, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, ...","[17, 13, 1, 1, 4, 19, 23, 23, 39, 23, 16, 1, 1..."
999996,999996,3999997,"[2218777, 55004, 188638, 33735, 265430, 3719, ...","[1018926, 8039, 350711, 153260, 467025, 2344, ...","[19738, 2291, 3024, 1369, 105, 10, 115, 5396, ...","[3, 315, 3, 72, 3, 3, 3, 688, 3, 3, 76, 3, 3, ...","[1, 2, 1, 2, 7, 1, 1, 2, 4, 1, 2, 4, 1, 1, 1, ...","[135, 111, 26, 118, 1, 1, 121, 20, 11, 71, 21,..."
999997,999997,3999998,"[20145, 64331, 1027, 1011104, 88711, 2974, 106...","[36243, 151377, 481, 955374, 8736, 691, 42069,...","[40, 904, 10, 1861, 904, 184, 1923, 2989, 8542...","[80, 3, 3, 57, 3, 7, 140, 238, 3, 57, 307, 57,...","[2, 1, 7, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 4, ...","[47, 20, 33, 23, 54, 1, 50, 17, 143, 23, 1, 23..."
999998,999998,3999999,"[5487, 30003, 5368, 16807, 1558, 2176946, 128,...","[1457, 108615, 1461, 5885, 15618, 492992, 1563...","[85, 3523, 587, 577, 2979, 4041, 581, 269, 577...","[83, 41, 3, 3, 3, 57, 3, 3, 3, 355, 3, 3, 57, ...","[2, 2, 1, 4, 1, 2, 7, 7, 4, 2, 1, 7, 2, 2, 2, ...","[4, 44, 23, 18, 8, 23, 13, 147, 34, 17, 1, 71,..."


In [4]:
test_dataset = MultTensorDataset(feat, df_seq_data_test)
test_dataloader = DataLoader(dataset = test_dataset,shuffle = False)

In [5]:
embedding = []
for f in feat:
    embedding.append(load_w2v(f))

./w2v_model/word2vec_creative_id_128_win100.model
Word2Vec(vocab=3412772, size=128, alpha=0.025)
(3412773, 128)
./w2v_model/word2vec_ad_id_128_win100.model
Word2Vec(vocab=3027360, size=128, alpha=0.025)
(3027361, 128)
./w2v_model/word2vec_advertiser_id_128_win100.model
Word2Vec(vocab=57870, size=128, alpha=0.025)
(57871, 128)
./w2v_model/word2vec_product_id_128_win100.model
Word2Vec(vocab=39057, size=128, alpha=0.025)
(39058, 128)


In [14]:
def test(model, device, dataloader):
    pred = []
    with torch.no_grad():
        for data in tqdm(dataloader,total=len(dataloader)):
            for i in range(len(data)):
                data[i] = data[i].to(device)
            output = model(data)
            p = output.max(1, keepdim = True)[1] 
            pred.append(p.item())
    
    return pred

In [7]:
embed_gender=[128,128,128,128]
hidden_gender=64
test_gender_model = ADLSTM_Gender__Model(2,pre_embed=embedding,embedding_dim=embed_gender,hidden_dim=hidden_gender,bidirectional=True)
test_gender_model.load_state_dict(torch.load(model_gender_path))

<All keys matched successfully>

In [35]:
embed_age=[128,128,128,128]
hidden_age=128
test_age_model = ADEM_Age_Model(10,pre_embed=embedding,embedding_dim=embed_age,hidden_dim=hidden_age,bidirectional=True)
test_age_model.load_state_dict(torch.load(model_age_path))

SingleAgeModel(
  (embedding): Embedding(3027361, 128)
  (lstm): LSTM(128, 128, num_layers=2, bidirectional=True)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


<All keys matched successfully>

In [ ]:
df_res = df_seq_data_test[['user_id']]
# df_res['predicted_age'] = test(test_age_model,DEVICE,test_dataloader)
df_res['predicted_gender'] = test(test_gender_model,DEVICE,test_dataloader)

In [12]:
df_res['predicted_gender'] = df_res['predicted_gender'] + 1
# df_res['predicted_age'] = df_res['predicted_age'] + 1
df_res.to_csv('submission_lstm_single.csv', index=False)

In [4]:
df_res1 = pd.read_csv('submission_multlstm_age.csv')
df_res2 = pd.read_csv('submission_multlstm_gender.csv')
df_res1['predicted_gender'] = df_res2['predicted_gender']
df_res1.to_csv('submission_multlstm2.csv', index=False)

In [5]:
df_res1

,user_id,predicted_age,predicted_gender
0,3000001,3,1
1,3000002,7,2
2,3000003,3,2
3,3000004,3,1
4,3000005,4,1
...,...,...,...
999995,3999996,3,1
999996,3999997,2,1
999997,3999998,2,1
999998,3999999,2,1
